pdb document to cif document

In [ ]:
import os
from Bio.PDB import PDBParser

# ===== Input multi-frame PDB file =====
input_pdb = "frames1.pdb"

# ===== Output directory =====
base_dir = os.path.dirname(os.path.abspath(input_pdb))
output_dir = os.path.join(
    base_dir,
    os.path.splitext(os.path.basename(input_pdb))[0] + "_xyzs"
)
os.makedirs(output_dir, exist_ok=True)

# ===== Parse multi-frame PDB =====
parser = PDBParser(QUIET=True)
structure = parser.get_structure("traj", input_pdb)

# ===== Iterate over each model (frame) =====
for i, model in enumerate(structure, start=1):
    atoms = list(model.get_atoms())
    natoms = len(atoms)

    output_xyz = os.path.join(output_dir, f"frame_{i:04d}.xyz")
    with open(output_xyz, "w") as f:
        # First line: number of atoms
        f.write(f"{natoms}\n")
        # Second line: comment
        f.write(f"Frame {i}\n")
        # Atom coordinates
        for atom in atoms:
            element = atom.element if atom.element != "" else atom.get_id()[0]
            x, y, z = atom.coord
            f.write(f"{element:2s} {x:10.5f} {y:10.5f} {z:10.5f}\n")

    print(f"Wrote {output_xyz}")

print(f"All done! XYZ files saved in {output_dir}")


convert .pdb to .cif

In [ ]:
# -*- coding: utf-8 -*-
import os
from ase.io import read, write

# Input and output folder (current directory)
input_folder = os.getcwd()
output_folder = os.getcwd()
os.makedirs(output_folder, exist_ok=True)

# Collect all PDB files in the input folder
pdb_files = [f for f in os.listdir(input_folder) if f.endswith(".pdb")]

for idx, pdb_file in enumerate(pdb_files, 1):
    input_path = os.path.join(input_folder, pdb_file)
    base_name = os.path.splitext(pdb_file)[0]

    # Read all frames from the PDB file
    frames = read(input_path, index=":")

    # Write each frame as an XYZ file
    for frame_idx, frame in enumerate(frames, 1):
        output_path = os.path.join(output_folder, f"{base_name}_frame{frame_idx:04d}.xyz")
        write(output_path, frame, format="xyz")

    print(f"[{idx}/{len(pdb_files)}] {pdb_file} divided into {len(frames)} .XYZ files")

print("\nResults saved in:", output_folder)


convert xyz to cif

In [ ]:
import os
from ase.io import read, write

# Input path
input_path = "frame_0001.xyz"

# Output path (same folder, change extension to .cif)
base, _ = os.path.splitext(input_path)
output_path = base + ".cif"

# Read xyz
atoms = read(input_path)

# Set box size 30 Å × 30 Å × 30 Å
atoms.set_cell([30, 30, 30])
atoms.set_pbc(True)   # enable periodic boundary conditions

# Write cif
write(output_path, atoms)

print(f"Converted and boxed: {input_path} → {output_path}")


.cif to .cell

In [ ]:
from ase.io import read, write
import os

# Input folder containing .cif files
input_folder = os.getcwd()

# Output folder for .cell files
output_folder = os.path.join(input_folder, "cell_files")
os.makedirs(output_folder, exist_ok=True)

# Loop over all files in the input folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".cif"):
        # Input file path
        filepath = os.path.join(input_folder, filename)
        
        # File name without extension
        documentname = os.path.splitext(filename)[0]
        
        # Output file path (saved in output_folder)
        output_path = os.path.join(output_folder, f"{documentname}.cell")
        
        try:
            # Read CIF file
            atoms = read(filepath)
            
            # Write CASTEP .cell file
            write(output_path, atoms, format="castep-cell")
            
            print(f"Converted: {filepath} → {output_path}")
        except Exception as e:
            print(f"Failed: {filepath}, error: {e}")

print(f"\nAll .cell files saved in: {output_folder}")
